# Lab: Credit Card Fraud Detection revised

we return to the use case from last week and apply Random Forests.


## Pre-Stage
Use your pre-processing from the last assignment as input.


In [1]:
#download data
#!unzip creditcard.csv.zip

#from zipfile import ZipFile
#zip = ZipFile('creditcard.csv.zip')
#zip.extractall()

In [10]:
# your preprocessing code5
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, normalize, RobustScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_recall_fscore_support, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn import preprocessing



In [11]:
A = pd.read_csv('creditcard.csv')

In [12]:
std_scaler = StandardScaler()
rob_scaler = RobustScaler()

A['scaled_amount'] = rob_scaler.fit_transform(A['Amount'].values.reshape(-1,1))
A['scaled_time'] = rob_scaler.fit_transform(A['Time'].values.reshape(-1,1))

A.drop(['Time','Amount'], axis=1, inplace=True)

scaled_amount = A['scaled_amount']
scaled_time = A['scaled_time']

A.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
A.insert(0, 'scaled_amount', scaled_amount)
A.insert(1, 'scaled_time', scaled_time)

# Amount and Time are Scaled!

A.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


In [14]:
X = A.drop('Class', axis=1)
y = A['Class']

sss = StratifiedKFold(n_splits=2, random_state=None, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    Xtrain, Xtest = X.iloc[train_index], X.iloc[test_index]
    ytrain, ytest = y.iloc[train_index], y.iloc[test_index]

Xtrain = Xtrain.values
Xtest = Xtest.values
ytrain = ytrain.values
ytest = ytest.values

train_unique_label, train_counts_label = np.unique(ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(ytest, return_counts=True)


print('Label Distributions: \n')
print(train_counts_label/ len(ytrain))
print(test_counts_label/ len(ytest))

Train: [119781 120505 120837 ... 284804 284805 284806] Test: [     0      1      2 ... 142425 142426 142427]
Train: [     0      1      2 ... 142425 142426 142427] Test: [119781 120505 120837 ... 284804 284805 284806]
Label Distributions: 

[0.99827252 0.00172748]
[0.99827251 0.00172749]


# Task: Train a Random Forest Classifier
* Train and evaluate the model 
* Compare the results to the Bayesian Classifier
* Tune the hyper-parameters (like number of trees, tree depth, ...) to optimize the results

See: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier

In [15]:
# Instantiate model with 100 decision trees
rf_model = RandomForestClassifier(class_weight='balanced', n_jobs=-1)
param_set = {
    'max_depth':[None, 10, 20],
    'n_estimators':[100, 50, 20],
    'min_samples_split':[2, 3],
    'max_features':['auto', 10, 5]
}

rf = RandomizedSearchCV(rf_model, param_set, n_iter=1,random_state=0, cv=3)
# Train the model on training data
rf.fit(Xtrain, ytrain)

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=1, n_jobs=1,
          param_distributions={'max_depth': [None, 10, 20], 'n_estimators': [100, 50, 20], 'min_samples_split': [2, 3], 'max_features': ['auto', 10, 5]},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [16]:
predictTrainLabel = rf.best_estimator_.predict(Xtrain)
print (classification_report(ytrain,predictTrainLabel))

print(accuracy_score(ytrain,predictTrainLabel))
#print(precision_recall_fscore_support(ytrain, predictTestLabel, labels=['1','0']))



             precision    recall  f1-score   support

          0       1.00      1.00      1.00    142158
          1       1.00      0.98      0.99       246

avg / total       1.00      1.00      1.00    142404

0.9999508440774135


In [17]:
predictTrainLabel = rf.best_estimator_.predict(Xtest)
print (classification_report(ytest,predictTrainLabel))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00    142157
          1       1.00      0.06      0.11       246

avg / total       1.00      1.00      1.00    142403



In [18]:
print(precision_recall_fscore_support(ytrain, predictTrainLabel, labels=['1','0']))

ValueError: Found input variables with inconsistent numbers of samples: [142404, 142403]